# 지도학습(4) - 회귀모델 성능 측정 지표

회귀분석을 통해 에측 모델을 만들고 해당 모델의 성능을 파악하기 위해 제공되는 사이킷런의 성능 측정 지표모듈

### #01. 작업준비

#### 패키지 가져오기

In [2]:
import sys

sys.path.append("../../")
import helper

import numpy as np
import seaborn as sb
from pandas import read_excel, DataFrame
from matplotlib import pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

# 성능 측정 지표 모듈
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error

### 샘플 회귀분석 모델 만들기
#### #02. 자동차 속도에 따른 제동거리 회귀모델
#### 데이터 가져오기

In [3]:
origin = read_excel("https://data.hossam.kr/E04/cars.xlsx")
origin.head()

,speed,dist
0,4,2
1,4,10
2,7,4
3,7,22
4,8,16


#### 독립변수에 대한 다항식 생성

In [4]:
poly = PolynomialFeatures(include_bias=False)
fit = poly.fit_transform(origin[['speed']])
x = DataFrame(fit, columns=poly.get_feature_names_out())
x.head()

,speed,speed^2
0,4.0,16.0
1,4.0,16.0
2,7.0,49.0
3,7.0,49.0
4,8.0,64.0


종속변수 추출

In [5]:
y = origin[['dist']]
y.head()

,dist
0,2
1,10
2,4
3,22
4,16


데이터 분할

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True, random_state=42)
x_train.shape, y_train.shape

((35, 2), (35, 1))

In [22]:
model = LinearRegression()
fit = model.fit(x_train, y_train)

print("계수: ", fit.coef_)
print("절편: ", fit.intercept_)
print("훈련 데이터 설명력: ", fit.score(x_train, y_train))
print("검증 데이터 설명력: ", fit.score(x_test, y_test))

계수:  [[1.97370235 0.0601573 ]]
절편:  [-4.44412881]
훈련 데이터 설명력:  0.6386048074085511
검증 데이터 설명력:  0.7101851923162426


예측값 생성

훈련 데이터에 대한 예측값

In [8]:
y_train_pred = fit.predict(x_train)
y_train_pred = y_train_pred.reshape(-1)
y_train_pred

array([21.30862466, 59.09283798, 77.57533206, 77.57533206, 31.38058538,
       24.5456303 , 31.38058538, 38.69679884, 50.57347853, 50.57347853,
        4.4131974 , 72.77423665, 42.53537747, 50.57347853, 18.19193362,
       46.4942707 , 27.90295054, 54.77300096,  4.4131974 , 34.97853481,
       12.31949534, 68.09345583, 54.77300096, 38.69679884, 59.09283798,
       24.5456303 , 34.97853481, 31.38058538, 82.49674208, 34.97853481,
       21.30862466, 59.09283798, 27.90295054, 46.4942707 , 59.09283798])

검증 데이터에 대한 예측값

In [9]:
y_test_pred = fit.predict(x_test)
y_test_pred = y_test_pred.reshape(-1)
y_test_pred

array([27.90295054, 59.09283798, 46.4942707 , 77.57533206, 31.38058538,
       77.57533206, 42.53537747, 38.69679884, 50.57347853, 34.97853481,
       27.90295054, 15.19555718, 54.77300096, 21.30862466, 12.31949534])

### #03. 회귀분석 모델의 성능평가

회귀분석 모델의 평가를 위한 지표는 실제값(관측치) 회귀 예측값의 차이를 기반으로 한다.(잔차를 기반으로)

|구분|설명|
|--|--|
|에러율|낮을 수록 좋음(0에 가까울수록)|
|설명력|높을 수록 좋음(1에 가까울수록)|


### 1) 설명력
### $R^2$ (결정계수)
회귀분석에서 가장 많이 채택되는 설명력 값

기본적으로 모델의 학습 결과를 갖고 있는 'fit'객체의 score() 매서드를 통해 조회가능

In [10]:
print("훈련 데이터 설명력: ", fit.score(x_train, y_train))
print("검증 데이터 설명력: ", fit.score(x_test, y_test))

훈련 데이터 설명력:  0.6386048074085511
검증 데이터 설명력:  0.7101851923162426


sklearn이 제공하는 metrics 객체에 r2_score() 매서드를 통해서도 조회할 수 있다.

이 때 파라미터는 관측치와 예측치를 전달한다.

In [11]:
print("훈련데이터 설명력 :", r2_score(y_train, y_train_pred))
print("검증데이터 설명력 :",r2_score(y_test, y_test_pred))

훈련데이터 설명력 : 0.6386048074085511
검증데이터 설명력 : 0.7101851923162426


#### 2) 에러율

|종류|이름|한글명|잔차계산|이상치 영향여부|
|--|--|--|--|--|
|MAE|Mean Abolute Error|평균절대오차|절대값|Yes|
|MSE|Mean Squared Error|평균제곱오차|제곱값|No|
|RMSE|Root Mean Squared Error|평균오차|제곱값|No|
|MAPE|Mean Abolute Percentage Error|평균 절대 백분 오차 비율|절대값|Yes|
|MPE|Mean Percentage Error|평균 비율 오차|N/A|Yes|

#### $MAE$ (mean, Absolute, Error) : 평균절대오차

![mae_expr](res/mae_expr.png)

모델의 예측값과 실제값의 차이를 모두 더하는 개념이다.(잔차 더하기)

절대값을 취하기 때문에 직관적으로 알 수 있는 지표다.

이상치에 영향을 받지 않는다.

MAE는 절대값을 취하는 지표이기에 실제보다 낮은 값(underperformance)인지 큰 값(overperformance)인지 알 수 없다.

실제 데이터를 기준으로 오차가 얼마날 것인지? ex) 11이면 관측치 13 실제값 2

![mae](res/mae.jpg)

In [12]:
print("훈련데이터 MAE :", mean_absolute_error(y_train, y_train_pred))
print("검증데이터 MAE :",mean_absolute_error(y_test, y_test_pred))

훈련데이터 MAE : 11.328321180738177
검증데이터 MAE : 10.378055983452917


#### $MSE$ (Mean Squared Error)

![img](res/mse_expr.png)

MAE와는 다르게 제곱을 하기 때문에 모델의 실제값과 예측값의 차이의 면적의 합이다.

제곱을 하기때문에 특이값이 존재하면 수치가 많이 늘어난다.(=특이치에 민감)

![img](res/mse.jpg)



In [13]:
print("훈련데이터 MSE :", mean_squared_error(y_train, y_train_pred))
print("검증데이터 MSE :",mean_squared_error(y_test, y_test_pred))
# 제곱근을 구해서 RMAE값을 구하는 것이 일반적

훈련데이터 MSE : 229.58182765278815
검증데이터 MSE : 197.51716386425815


#### $RMSE$ (Root Mean Squared Error) : 평균 오차 

![rmse](res/rmse_expr.png)

MSE를 구한 값에 루트를 씌운다.

오류 지표를 실제 값과 유사한 단위로 변환하여 해석을 쉽게 한다.

###### np.sqrt 값의 루트 씌우기

In [14]:
print("훈련데이터 RMSE :", np.sqrt(mean_squared_error(y_train, y_train_pred)))
print("검증데이터 RMSE :", np.sqrt(mean_squared_error(y_test, y_test_pred)))

훈련데이터 RMSE : 15.151957881831251
검증데이터 RMSE : 14.05407997217385


#### $MAPE$ (Mean Abolute Percentage Error) : 평균 절대 백분 오차 비율

![img](res/mape_expr.png)

MAE를 퍼센트로 변환한 것이다.

MAE와 동일하게 MSE보다 이상치에 민감하며 실제값보다 낮은 값인지 높은 값인지 알 수없다.

모델에 대한 편향이 있다. (이를 대응하기 위해 MPE도 추가로 확인하는 것을 추천)

![img](res/mape.jpg)



In [15]:
# API로 제공되는 기능이 아니고, 직접 계산해야 하기 때문에 관측치와 예측치의 데이터 타입이 일치해야 한다.
# -> numpy 배열 혹은 Series 타입으로 통일해야 한다.
print("훈련 데이터 MAPE: ", np.mean(np.abs((y_train.values - y_train_pred) / y_train.values)) * 100)
print("검증 데이터 MAPE: ", np.mean(np.abs((y_test.values - y_test_pred) / y_test.values)) * 100)

훈련 데이터 MAPE:  160.26058863689647
검증 데이터 MAPE:  65.26193942487288


#### $MPE$ (Mean Percent Error) : 평균 비율 오차

![img](res/mpe_expr.png)

MAPE와 비슷하지만 MAPE에서 절대값을 제외한 지표다.

장점은 모델이 실제값보다 낮은 값인지 큰 값인지 판단 할 수 있다.

![img](res/mpe.jpg)


In [16]:
# API로 제공되는 기능이 아니고,
# 직접 계산해야 하기 때문에 관측치와 데이터 타입이 일치해야 한다.
# -> numpy 배열 혹은 Series 타입으로 통일해야 한다.
# print("훈련데이터 MAPE :", np.mean((np.absmean_squared_error(y_train.values - y_train_pred.values) / y_train.values * 100))
# print("검증데이터 MAPE :", np.mean((np.absmean_squared_error(y_test.values - y_test_pred.values) / y_test.values * 100))

In [17]:
# API로 제공되는 기능이 아니고, 직접 계산해야 하기 때문에 관측치와 예측치의 데이터 타입이 일치해야 한다.
# -> numpy 배열 혹은 Series 타입으로 통일해야 한다.
print("훈련 데이터 MPE: ", np.mean((y_train.values - y_train_pred) / y_train.values) * 100)
print("검증 데이터 MPE: ", np.mean((y_test.values - y_test_pred) / y_test.values) * 100)

훈련 데이터 MPE:  -118.10435798524273
검증 데이터 MPE:  -21.652277000596218


#### #04. 머신러닝 회귀분석 모듈 테스트
회귀분석 수행

In [18]:
olsResult = helper.ml_ols(origin, xnames='speed', yname='dist', degree=2, test_size=0.3, random_state=42)
print("계수:", olsResult.coef)
print("절편:", olsResult.intercept)

계수: [[1.97370235 0.0601573 ]]
절편: [-4.44412881]


훈련데이터에 대한 평가지표

In [19]:
print("R^2: ", olsResult.trainRegMetric.r2)
print("MAE: ", olsResult.trainRegMetric.mae)
print("MSE: ", olsResult.trainRegMetric.mse)
print("RMSE: ", olsResult.trainRegMetric.rmse)
print("MAPE: ", olsResult.trainRegMetric.mape)
print("MPE: ", olsResult.trainRegMetric.mpe)

R^2:  0.6386048074085511
MAE:  11.328321180738177
MSE:  229.58182765278815
RMSE:  15.151957881831251
MAPE:  39.42700152257923
MPE:  -19.82562327875809


검증 데이터에 대한 평가지표

In [21]:
print("R^2: ", olsResult.testRegMetric.r2)
print("MAE: ", olsResult.testRegMetric.mae)
print("MSE: ", olsResult.testRegMetric.mse)
print("RMSE: ", olsResult.testRegMetric.rmse)
print("MAPE: ", olsResult.testRegMetric.mape)
print("MPE: ", olsResult.testRegMetric.mpe)

R^2:  0.7101851923162426
MAE:  10.378055983452917
MSE:  197.51716386425815
RMSE:  14.05407997217385
MAPE:  23.404866814977726
MPE:  2.44954555043573
